In [ ]:
!apt-get install unrar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal

Cannot open /content/drive/MyDrive/Colab.rar
No such file or directory
No files to extract


In [ ]:
!unrar x "/content/drive/MyDrive/Colab Notebooks/samsa_dataset/samsa_dataset.rar" "/content/drive/MyDrive/Colab Notebooks/samsa_dataset/samsa_dataset/"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
img_path = '/content/drive/MyDrive/Colab Notebooks/dataset/samsa_dataset/'

In [ ]:
train_path = img_path + 'training/'
test_path = img_path + 'test/'

In [ ]:
!pip install scikeras

In [ ]:
import os

import numpy as np
import pandas as pd

import keras
from keras.layers import Dense, Flatten
from keras.models import Sequential
from scikeras.wrappers import KerasClassifier
from keras.datasets import mnist
from keras.utils import to_categorical

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPool2D, Dropout
from keras.applications import Xception
# from keras.applications.vgg16 import VGG16
from keras.callbacks import EarlyStopping

from tensorflow.keras.regularizers import l2
from tensorflow.keras.regularizers import l1

from keras.layers import GlobalAveragePooling2D
from keras.utils import plot_model

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_gen_aug = ImageDataGenerator(
    rescale = 1/255.,
    shear_range=0.1,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip=True
)

In [ ]:
test_gen_aug = ImageDataGenerator(
    rescale = 1/255.,
    shear_range=0.1,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip=True
)

In [ ]:
train_aug = train_gen_aug.flow_from_directory(train_path,
        target_size=(299, 299),
        class_mode=None,
        shuffle=False
)


Found 2905 images belonging to 2 classes.


In [ ]:
test_aug = test_gen_aug.flow_from_directory(test_path,
        target_size=(299, 299),
        class_mode=None,
        shuffle=False
)


Found 368 images belonging to 2 classes.


In [ ]:
print(train_aug.class_indices)
print(test_aug.class_indices)


{'not_samsa': 0, 'samsa': 1}
{'not_samsa': 0, 'samsa': 1}


In [ ]:
conv_base = Xception(include_top=False, input_shape=(299, 299, 3))

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Running Model: 1


In [ ]:
X_train = conv_base.predict(train_aug)
X_test = conv_base.predict(test_aug)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


91/91 ━━━━━━━━━━━━━━━━━━━━ 107s 949ms/step
12/12 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step


In [ ]:
y_train = train_aug.labels
y_train_binarized = to_categorical(y_train)

y_test = test_aug.labels
y_test_binarized = to_categorical(y_test)

In [ ]:
model = Sequential()

model.add(GlobalAveragePooling2D())


# model.add(Dense(2, activation='sigmoid'))
model.add(Dense(2, activation='softmax'))
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy', 'Precision', 'Recall']
)


In [ ]:
callback = EarlyStopping(patience=5)

history = model.fit(
        X_train,
        y_train_binarized,
        epochs = 20,
        validation_data=(X_test, y_test_binarized),
        callbacks=[callback],
        verbose=1
)

Epoch 1/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - Precision: 0.8450 - Recall: 0.8450 - accuracy: 0.8450 - loss: 0.3098 - val_Precision: 0.9565 - val_Recall: 0.9565 - val_accuracy: 0.9565 - val_loss: 0.1620
Epoch 2/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - Precision: 0.9615 - Recall: 0.9615 - accuracy: 0.9615 - loss: 0.1185 - val_Precision: 0.9565 - val_Recall: 0.9565 - val_accuracy: 0.9565 - val_loss: 0.1256
Epoch 3/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - Precision: 0.9719 - Recall: 0.9719 - accuracy: 0.9719 - loss: 0.0856 - val_Precision: 0.9565 - val_Recall: 0.9565 - val_accuracy: 0.9565 - val_loss: 0.1119
Epoch 4/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - Precision: 0.9741 - Recall: 0.9741 - accuracy: 0.9741 - loss: 0.0690 - val_Precision: 0.9592 - val_Recall: 0.9592 - val_accuracy: 0.9592 - val_loss: 0.1051
Epoch 5/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - Precision: 0.9869 - Recall: 0.9869 - accuracy: 0.9869 - loss: 0.0547 - val_Precision: 0.9620 - val_Recall: 0.96

In [ ]:
save_path = f'/content/drive/MyDrive/Colab Notebooks/new_model/samsa_model_1ver20.h5'
model.save(save_path)
print(f'File saved to: {save_path}')

File saved to: /content/drive/MyDrive/Colab Notebooks/new_model/samsa_model_1ver20.h5


In [ ]:
from keras.models import load_model


model_path = f'/content/drive/MyDrive/Colab Notebooks/new_model/samsa_model_1.h5'
model = load_model(model_path)

test_loss, test_accuracy, test_precision, test_recall = model.evaluate(X_test, y_test_binarized)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")


6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 131ms/step - accuracy: 0.9283 - loss: 0.1702 - precision_1: 0.9283 - recall_1: 0.9283
Test Loss: 0.13520613312721252
Test Accuracy: 0.9404761791229248
Test Precision: 0.9404761791229248
Test Recall: 0.9404761791229248
